In [22]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests

starbucks 사이트에서 시도 목록을 얻어와서 starbucks 데이터베이스의 sido 컬렉션에 저장한다.

In [23]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
sido = starbucks['sido']
sido.delete_many({})

request = requests.post('https://www.starbucks.co.kr/store/getSidoList.do').json()
sidoList = []
for r in request['list']:
    d = {'sido_cd': r['sido_cd'].strip(), 'sido_nm': r.get('sido_nm').strip()}
    sidoList.append(d)
    
for s in sidoList:
    sido.insert_one(s)

시도 이름을 입력받아 시도의 구군 코드 정보를 얻어와서 starbucks 데이터베이스의 gugun 컬렉션에 저장한다.

In [24]:
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
for s in sidoList:
    print('[{}]'.format(s['sido_nm']), end=' ')

sidoName = input('\n구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0}).next()

request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', 
            data={
                'sido_cd': sido_cd['sido_cd']
            }).json()
gugunList = []
for r in request['list']:
    if r.get('gugun_nm') is None:
        r['gugun_nm'] = '세부지역없음'
    d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r.get('gugun_nm').strip()}
    gugunList.append(d)
    
gugun = starbucks['gugun']
gugun.delete_many({})
for s in gugunList:
    gugun.insert_one(s)
    
gugunList = gugun.find({}, {'_id': 0})
for g in gugunList:
    print('[{}]'.format(g['gugun_nm']), end=' ')

[서울] [경기] [광주] [대구] [대전] [부산] [울산] [인천] [강원] [경남] [경북] [전남] [전북] [충남] [충북] [제주] [세종] 
구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 경기
[가평군] [고양시 덕양구] [고양시 일산동구] [고양시 일산서구] [과천시] [광명시] [광주시] [구리시] [군포시] [김포시] [남양주시] [동두천시] [부천시] [성남시 분당구] [성남시 수정구] [성남시 중원구] [수원시 권선구] [수원시 영통구] [수원시 장안구] [수원시 팔달구] [시흥시] [안산시 단원구] [안산시 상록구] [안성시] [안양시 동안구] [안양시 만안구] [양주시] [양평군] [여주시] [연천군] [오산시] [용인시 기흥구] [용인시 수지구] [용인시 처인구] [의왕시] [의정부시] [이천시] [파주시] [평택시] [포천시] [하남시] [화성시] 

시도 이름과 구군 이름을 입력받아 매장 정보를 얻어와서 starbucks 데이터베이스의 store 컬렉션에 저장한다.

In [26]:
gugunName = input('\n구군별 매장 정보를 검색하려는 구군 이름을 입력하세요(예: 강남구): ')
gugun = starbucks['gugun']
gugun_cd = gugun.find({'gugun_nm': gugunName}).next()
# print(gugun_cd)

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=7ZVMNDIP9E', 
            data={
                'ins_lat': 37.563398,
                'ins_lng': 126.9863309,
                'p_sido_cd': '',
                'p_gugun_cd': gugun_cd['gugun_cd'],
                'in_biz_cd': '',
                'iend': 2000,
                'set_date': ''
            }).json()
storeList = []
for s in request['list']:
    d = {'s_name': s['s_name'].strip(), 'doro_address': s['doro_address'].strip(), 'lat': s['lat'], 'lot': s['lot']}
    # print(d)
    storeList.append(d)
# print(storeList)

store = starbucks['store']
store.delete_many({})
for s in storeList:
    store.insert_one(s)
    
storeList = store.find({}, {'_id': 0})
for s in storeList:
    print(s)


구군별 매장 정보를 검색하려는 구군 이름을 입력하세요(예: 강남구): 양주시
{'s_name': '양주회정DT', 'doro_address': '경기도 양주시 평화로 1687 (회정동)', 'lat': '37.8394993554251', 'lot': '127.05474819186'}
{'s_name': '양주옥정공원', 'doro_address': '경기도 양주시 옥정동로7길 30 (옥정동)', 'lat': '37.8212389157803', 'lot': '127.097494414027'}
{'s_name': '양주고읍', 'doro_address': '경기도 양주시 고읍남로 20 (광사동)', 'lat': '37.794628', 'lot': '127.079196'}
{'s_name': '양주광사DT', 'doro_address': '경기도 양주시 부흥로 1872 (광사동)', 'lat': '37.790532', 'lot': '127.072421'}
{'s_name': '양주고읍DT', 'doro_address': '경기도 양주시 고덕로351 (고읍동)', 'lat': '37.803684', 'lot': '127.079474'}
{'s_name': '양주덕정역DT', 'doro_address': '경기도 양주시 독바위로 35 (덕정동)', 'lat': '37.8382562', 'lot': '127.062968'}
{'s_name': '양주송추DT', 'doro_address': '경기도 양주시 장흥면 호국로 528', 'lat': '37.717529', 'lot': '126.973093'}
{'s_name': '양주옥정타운', 'doro_address': '경기도 양주시 회천남로 76 (옥정동)', 'lat': '37.81858', 'lot': '127.094293'}


시도 이름을 입력받아 시도 전체 매장 정보를 starbucks 데이터베이스의 store_sido 컬렉션에 저장한다.

In [42]:
sidoName = input('\n구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido = starbucks['sido']
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0}).next()
# print(sido_cd)

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=7ZVMNDIP9E', 
            data={
                'ins_lat': 37.563398,
                'ins_lng': 126.9863309,
                'p_sido_cd': sido_cd['sido_cd'],
                'p_gugun_cd': '',
                'in_biz_cd': '',
                'iend': 2000,
                'set_date': ''
            }).json()
storeList = []
for s in request['list']:
    d = {'s_name': s['s_name'].strip(), 'doro_address': s['doro_address'].strip(), 'lat': s['lat'], 'lot': s['lot']}
    # print(d)
    storeList.append(d)
# print(storeList)

store_sido = starbucks['store_sido']
store_sido.delete_many({})
for s in storeList:
    store_sido.insert_one(s)
    
storeList = store_sido.find({}, {'_id': 0})
for s in storeList:
    print(s)


구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 세종
{'s_name': '세종다정DT', 'doro_address': '세종특별자치시 다정북로 219 (다정동)', 'lat': '36.494878', 'lot': '127.255384'}
{'s_name': '세종보람', 'doro_address': '세종특별자치시 한누리대로 2165 (보람동) A118,119,120,121,128,129,130,131', 'lat': '36.4773322231936', 'lot': '127.28651937895552'}
{'s_name': '세종새롬', 'doro_address': '세종특별자치시  새롬중앙로 64 (새롬동)', 'lat': '36.486896', 'lot': '127.250718'}
{'s_name': '세종어진R', 'doro_address': '세종특별자치시  한누리대로 499 (어진동)', 'lat': '36.508188', 'lot': '127.261188'}
{'s_name': '세종종촌', 'doro_address': '세종특별자치시  도움3로 105-10 (종촌동) 성운프라자', 'lat': '36.5041123', 'lot': '127.2493812'}
{'s_name': '세종첫마을', 'doro_address': '세종특별자치시  한누리대로 169 행복타워 1층', 'lat': '36.4803826', 'lot': '127.2617948'}
{'s_name': '세종이마트', 'doro_address': '세종특별자치시 금송로 687(가람동 406) 이마트 세종점', 'lat': '36.4712189', 'lot': '127.2509973'}
{'s_name': '세종청사', 'doro_address': '세종특별자치시  도움8로 81 (어진동)', 'lat': '36.504565', 'lot': '127.270769'}
{'s_name': '세종아름', 'doro_address': '세종특별자치시 아름서1길 13-9 (아름

전국 전체 매장 정보를 starbucks 데이터베이스의 store_all 컬렉션에 저장한다.

In [43]:
request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=7ZVMNDIP9E', 
            data={
                'ins_lat': 37.563398,
                'ins_lng': 126.9863309,
                'p_sido_cd': '',
                'p_gugun_cd': '',
                'in_biz_cd': '',
                'iend': 2000,
                'set_date': ''
            }).json()
storeList = []
for s in request['list']:
    d = {'s_name': s['s_name'].strip(), 'doro_address': s['doro_address'].strip(), 'lat': s['lat'], 'lot': s['lot']}
    storeList.append(d)

store_all = starbucks['store_all']
store_all.delete_many({})
for s in storeList:
    store_all.insert_one(s)
    
storeList = store_all.find({}, {'_id': 0})
for s in storeList:
    print(s)

{'s_name': '구미인의', 'doro_address': '경상북도 구미시 인동중앙로 15 (인의동)', 'lat': '36.1069986', 'lot': '128.4207103'}
{'s_name': '홍대역', 'doro_address': '서울특별시 마포구 양화로 165, 상진빌딩 1층 (동교동)', 'lat': '37.557175', 'lot': '126.923654'}
{'s_name': '제주노형공원', 'doro_address': '제주특별자치도 제주시 연북로 12 (노형동)', 'lat': '33.481794', 'lot': '126.482523'}
{'s_name': '송파마천사거리DT', 'doro_address': '서울특별시 송파구 거마로 78 (마천동)', 'lat': '37.499829', 'lot': '127.147672'}
{'s_name': '그랜드조선 부산', 'doro_address': '부산광역시 해운대구 해운대해변로 292 (중동)', 'lat': '35.1600338', 'lot': '129.1630512'}
{'s_name': '서울웨이브아트센터', 'doro_address': '서울특별시 서초구 잠원로 145-35 (잠원동)', 'lat': '37.518807', 'lot': '127.00704'}
{'s_name': '명동남산', 'doro_address': '서울특별시 중구 퇴계로 132 (남산동3가)', 'lat': '37.560809', 'lot': '126.986397'}
{'s_name': '센텀몰1F', 'doro_address': '부산광역시 해운대구 센텀4로 15 (우동)', 'lat': '35.16981023', 'lot': '129.12778297'}
{'s_name': '송도컨벤시아대로DT', 'doro_address': '인천광역시 연수구 벤처로 10 (송도동)', 'lat': '37.3824512554796', 'lot': '126.636816667557'}
{'s_name': '세종충남